In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import xarray as xr

from osgeo import gdal
#from gistool import rasterize
import pyhdf
from pyhdf.SD import SD
import geopandas as gpd

from scipy import stats
from scipy import integrate
from numpy import exp
import math
from pymannkendall import original_test as mk

import matplotlib as mpl
import matplotlib.pyplot as plt
## set the line width of the hatch
mpl.rcParams['hatch.linewidth'] = 0.5
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
import matplotlib.ticker as mticker

In [3]:
def cal_3month_ano_aver(ds1, mon1, mon2, mon3):
    ds_m0 = ds1.sel(time=ds1['time.month'].isin([mon1, mon2, mon3]))
    ds_m1 = ds_m0.groupby('time.year').mean(dim='time',skipna=True)
    ds_m = ds_m1['vari'].values

    anos = np.zeros_like(ds_m)

    for i in range(0,ds_m.shape[2]):
        for j in range(0,ds_m.shape[1]):
            v1 = ds_m[:,j,i]
            date = np.arange(2002,2023)
            
            non_nan_count = np.count_nonzero(~np.isnan(v1))
            if non_nan_count > 1:
                tr = mk(v1, alpha=0.05).trend
                if tr=='increasing':
                    pvalue=1
                    slo = stats.linregress(date,v1).slope
                    interc = stats.linregress(date,v1).intercept
                    trend = slo*date + interc
                    anos[:,j,i] = (v1 - trend)/trend

                elif tr=='decreasing':
                    pvalue=-1
                    slo = stats.linregress(date,v1).slope
                    interc = stats.linregress(date,v1).intercept
                    trend = slo*date + interc
                    anos[:,j,i] = (v1 - trend)/trend
                elif tr=='no trend':
                    pvalue=np.nan
                    anos[:,j,i] = (v1 - v1.mean())/v1.mean()
            else:
                anos[:,j,i] = (v1 - v1.mean())/v1.mean()
    
    times = pd.date_range('2002','2023',freq='Y')
    ds_ano = xr.Dataset({'vari': (['time','lat','lon'], anos)},
                        coords={'time': (['time'], times), 
                                'lat': (['lat'], ds_m0.lat.values),
                        'lon': (['lon'], ds_m0.lon.values),})
    return ds_ano

In [12]:
def cal_3month_ano_sum(ds1, mon1, mon2, mon3):
    ds_m0 = ds1.sel(time=ds1['time.month'].isin([mon1, mon2, mon3]))
    ds_m1 = ds_m0.groupby('time.year').sum(dim='time',skipna=True)
    ds_m = ds_m1['vari'].values

    anos = np.zeros_like(ds_m)

    for i in range(0,ds_m.shape[2]):
        for j in range(0,ds_m.shape[1]):
            v1 = ds_m[:,j,i]
            date = np.arange(2002,2023)
            
            non_nan_count = np.count_nonzero(~np.isnan(v1))
            if non_nan_count > 1:
                tr = mk(v1, alpha=0.05).trend
                if tr=='increasing':
                    pvalue=1
                    slo = stats.linregress(date,v1).slope
                    interc = stats.linregress(date,v1).intercept
                    trend = slo*date + interc
                    anos[:,j,i] = (v1 - trend)/trend

                elif tr=='decreasing':
                    pvalue=-1
                    slo = stats.linregress(date,v1).slope
                    interc = stats.linregress(date,v1).intercept
                    trend = slo*date + interc
                    anos[:,j,i] = (v1 - trend)/trend
                elif tr=='no trend':
                    pvalue=np.nan
                    anos[:,j,i] = (v1 - v1.mean())/v1.mean()
            else:
                anos[:,j,i] = (v1 - v1.mean())/v1.mean()
    
    times = pd.date_range('2002','2023',freq='Y')
    ds_ano = xr.Dataset({'vari': (['time','lat','lon'], anos)},
                        coords={'time': (['time'], times), 
                                'lat': (['lat'], ds_m0.lat.values),
                        'lon': (['lon'], ds_m0.lon.values),})
    return ds_ano

In [2]:
def varimask_forest(ds):
    mask = xr.open_dataset('/portal1/dell/li-b/multiregression/LC_IGBP_2022_epsg4326.tif')

    x = ds.dims['lon']
    y = ds.dims['lat']
    qc_a = mask['band_data'].sel(band=1).values
    qc_b = np.zeros((y, x), dtype=bool)
    qc_b = (qc_a==1) | (qc_a == 2) | (qc_a == 4) | (qc_a == 5)
    anos = np.zeros_like(ds['vari'].values)
    for t in range(0,ds_ano789.dims['time']):    #### change the time length
        vari_a = ds['vari'][t,:,:].values
        vari_b = np.zeros((y, x))
        vari_b = np.where(qc_b, vari_a, np.nan)
        arr = vari_b.copy()##### exclude sichuan mountaions 
        arr[:80,:51] = np.nan
        anos[t,:,:] = arr
        
    ds_ex = xr.Dataset({'vari': (['time','lat','lon'], anos)},
                                coords={ 'time': (['time'], ds.time.values),
                                        'lat': (['lat'], ds.lat.values),
                                        'lon': (['lon'], ds.lon.values)})
        
    return(ds_ex)

In [4]:
def varimask_crop(ds):
    mask = xr.open_dataset('/portal1/dell/li-b/multiregression/LC_IGBP_2022_epsg4326.tif')

    x = ds.dims['lon']
    y = ds.dims['lat']
    qc_a = mask['band_data'].sel(band=1).values
    qc_b = np.zeros((y, x), dtype=bool)
    qc_b = (qc_a == 12) | (qc_a == 14)
    anos = np.zeros_like(ds['vari'].values)
    for t in range(0,ds_ano789.dims['time']):    
        vari_a = ds['vari'][t,:,:].values
        vari_b = np.zeros((y, x))
        vari_b = np.where(qc_b, vari_a, np.nan)
        anos[t,:,:] = vari_b
        
    ds_ex = xr.Dataset({'vari': (['time','lat','lon'], anos)},
                                coords={ 'time': (['time'], ds.time.values),
                                        'lat': (['lat'], ds.lat.values),
                                        'lon': (['lon'], ds.lon.values)})
        
    return(ds_ex)

In [11]:
ds1 = xr.open_dataset('/portal1/dell/li-b/multiregression/ndvi1.nc')
ds_ano789 = cal_3month_ano_aver(ds1, 7, 8, 9)
ds_fo = varimask_forest(ds_ano789)
ds_co = varimask_crop(ds_ano789)
to = ds_ano789['vari'].mean(dim=['lat','lon'],skipna=True).values
fo = ds_fo['vari'].mean(dim=['lat','lon'],skipna=True).values
co = ds_co['vari'].mean(dim=['lat','lon'],skipna=True).values
times = pd.date_range('2002','2023',freq='Y')
df1 = pd.DataFrame({'time':times,'to':to, 'fo':fo, 'co':co})
df1

,time,to,fo,co
0,2002-12-31,-0.000475,0.009768,-0.020157
1,2003-12-31,-0.035280,-0.013261,-0.056360
2,2004-12-31,-0.008093,-0.008044,-0.006185
3,2005-12-31,-0.000674,-0.002103,0.004433
4,2006-12-31,-0.002270,-0.004029,-0.002070
5,2007-12-31,0.005810,0.001521,0.004959
6,2008-12-31,0.004070,-0.017802,0.024943
7,2009-12-31,0.003991,-0.005430,0.014891
8,2010-12-31,0.006152,-0.005575,0.018150
9,2011-12-31,0.013895,-0.000807,0.037329


In [13]:
ds1 = xr.open_dataset('/portal1/dell/li-b/multiregression/lai1.nc') # already *0.1
ds_ano789 = cal_3month_ano_aver(ds1, 7, 8, 9)
ds_fo = varimask_forest(ds_ano789)
ds_co = varimask_crop(ds_ano789)
to = ds_ano789['vari'].mean(dim=['lat','lon'],skipna=True).values
fo = ds_fo['vari'].mean(dim=['lat','lon'],skipna=True).values
co = ds_co['vari'].mean(dim=['lat','lon'],skipna=True).values
times = pd.date_range('2002','2023',freq='Y')
df1 = pd.DataFrame({'time':times,'to':to, 'fo':fo, 'co':co})
df1

/home/li-b/.conda/envs/lbn/lib/python3.10/site-packages/xarray/core/concat.py:544: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


,time,to,fo,co
0,2002-12-31,-0.024760,-0.047785,-0.035898
1,2003-12-31,-0.055228,0.040784,-0.152115
2,2004-12-31,-0.021849,-0.022180,0.001372
3,2005-12-31,-0.019530,-0.036156,-0.008737
4,2006-12-31,0.007743,0.018444,-0.008868
5,2007-12-31,0.024379,-0.009570,0.052113
6,2008-12-31,0.002392,-0.018769,0.020178
7,2009-12-31,-0.008134,-0.016590,-0.012659
8,2010-12-31,0.016662,-0.017239,0.041153
9,2011-12-31,0.006137,0.003770,0.005055


In [14]:
ds1 = xr.open_dataset('/portal1/dell/li-b/multiregression/sif1.nc') 
ds_ano789 = cal_3month_ano_aver(ds1, 7, 8, 9)
ds_fo = varimask_forest(ds_ano789)
ds_co = varimask_crop(ds_ano789)
to = ds_ano789['vari'].mean(dim=['lat','lon'],skipna=True).values
fo = ds_fo['vari'].mean(dim=['lat','lon'],skipna=True).values
co = ds_co['vari'].mean(dim=['lat','lon'],skipna=True).values
times = pd.date_range('2002','2023',freq='Y')
df1 = pd.DataFrame({'time':times,'to':to, 'fo':fo, 'co':co})
df1

/home/li-b/.conda/envs/lbn/lib/python3.10/site-packages/xarray/core/concat.py:544: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


,time,to,fo,co
0,2002-12-31,-0.005528,0.000146,-0.006588
1,2003-12-31,0.050834,0.566466,-0.044983
2,2004-12-31,-0.021665,-0.013875,-0.041645
3,2005-12-31,-0.009036,0.007225,-0.019728
4,2006-12-31,0.022871,0.024316,0.032571
5,2007-12-31,-0.012742,-0.047882,-0.010829
6,2008-12-31,0.004561,0.019975,0.000932
7,2009-12-31,0.028473,0.035433,0.020125
8,2010-12-31,0.008287,0.018353,0.009571
9,2011-12-31,0.004534,0.001072,0.002100


In [10]:
ds1 = xr.open_dataset('/portal1/dell/li-b/multiregression/pre1.nc')
ds_ano789 = cal_3month_ano_sum(ds1, 7, 8, 9)
ds_fo = varimask_forest(ds_ano789)
ds_co = varimask_crop(ds_ano789)
to = ds_ano789['vari'].mean(dim=['lat','lon'],skipna=True).values
fo = ds_fo['vari'].mean(dim=['lat','lon'],skipna=True).values
co = ds_co['vari'].mean(dim=['lat','lon'],skipna=True).values
times = pd.date_range('1961','2023',freq='Y')
df1 = pd.DataFrame({'time':times,'to':to, 'fo':fo, 'co':co})
df1

/home/li-b/.conda/envs/lbn/lib/python3.10/site-packages/xarray/core/concat.py:544: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/tmp/ipykernel_158223/2166516367.py:31: RuntimeWarning: divide by zero encountered in divide
  anos[:,j,i] = (v1 - v1.mean())/v1.mean()
/tmp/ipykernel_158223/2166516367.py:31: RuntimeWarning: invalid value encountered in divide
  anos[:,j,i] = (v1 - v1.mean())/v1.mean()


,time,to,fo,co
0,1961-12-31,0.141619,0.461960,-0.089881
1,1962-12-31,0.118945,-0.091580,0.299831
2,1963-12-31,0.025619,-0.148425,0.250313
3,1964-12-31,-0.241951,-0.288688,-0.176704
4,1965-12-31,-0.020825,-0.181731,0.190942
...,...,...,...,...
57,2018-12-31,0.018112,0.121716,-0.044468
58,2019-12-31,-0.147453,0.047582,-0.372987
59,2020-12-31,0.443085,0.316462,0.476919
60,2021-12-31,0.170719,-0.068067,0.368959


In [13]:
ds1 = xr.open_dataset('/portal1/dell/li-b/multiregression/et1.nc')
ds_ano789 = cal_3month_ano_sum(ds1, 7, 8, 9)
ds_fo = varimask_forest(ds_ano789)
ds_co = varimask_crop(ds_ano789)
to = ds_ano789['vari'].mean(dim=['lat','lon'],skipna=True).values
fo = ds_fo['vari'].mean(dim=['lat','lon'],skipna=True).values
co = ds_co['vari'].mean(dim=['lat','lon'],skipna=True).values
times = pd.date_range('2002','2023',freq='Y')
df1 = pd.DataFrame({'time':times,'to':to, 'fo':fo, 'co':co})
df1

/home/li-b/.conda/envs/lbn/lib/python3.10/site-packages/xarray/core/concat.py:544: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/tmp/ipykernel_158223/3246300517.py:31: RuntimeWarning: divide by zero encountered in divide
  anos[:,j,i] = (v1 - v1.mean())/v1.mean()
/tmp/ipykernel_158223/3246300517.py:31: RuntimeWarning: invalid value encountered in divide
  anos[:,j,i] = (v1 - v1.mean())/v1.mean()


,time,to,fo,co
0,2002-12-31,-0.022446,-0.018483,-0.035724
1,2003-12-31,0.008422,0.044548,0.001009
2,2004-12-31,-0.046684,-0.030156,-0.042368
3,2005-12-31,0.008205,0.013745,0.009179
4,2006-12-31,-0.021879,-0.022869,-0.020174
5,2007-12-31,0.006740,-0.012699,0.031132
6,2008-12-31,-0.004790,-0.023614,0.010477
7,2009-12-31,0.013085,0.020744,-0.002073
8,2010-12-31,0.030134,-0.013597,0.051733
9,2011-12-31,-0.017568,-0.001780,-0.045408
